# Near shift-invariance in the Interpolated DWT (IDWT) domain

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mimg
%matplotlib inline
import pywt

In [ ]:
%%bash

# Create the two images ("/tmp/moving_circle_000.png" and "/tmp/moving_circle_001.png") with a circle
# of diameter 10 that moves from coordinate (20,30) to (21,30).
/home/vruiz/MRVC/tools/moving_circle.sh -w 128 -h 128 -x 20 -y 30 -d 10 -f 2
mv /tmp/moving_circle_000.png .
mv /tmp/moving_circle_001.png .

In [ ]:
def normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    return (img-min_component)/max_min_component

def show_img(prefix):
    img = load_img(prefix)
    img = normalize(img)
    plt.figure(figsize=(10,10))
    plt.title(prefix, fontsize=20)
    plt.imshow(img)
    
def load_frame(prefix):
    fn = f"{prefix}.png"
    print(fn)
    frame = cv2.imread(fn, cv2.IMREAD_UNCHANGED) # [rows, columns, components]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.array(frame)
    frame = frame.astype(np.float32) - 32768.0
    return frame

def write_frame(frame, prefix):
    frame = frame.astype(np.float32)
    frame += 32768.0
    frame = frame.astype(np.uint16)
    cv2.imwrite(f"{prefix}.png", frame)

def show_seq(prefix="/tmp/", frames=5):
    for i in range(frames):
        show_img(f"{prefix}{i:03d}")

def show_motion(prefix="/tmp/", frames=5):
    plt.figure(figsize=(10,10))
    prev = load_frame(f"{prefix}000")
    imgplot = plt.imshow(normalize(prev))
    for i in range(1, frames, 1):
        img = load_frame(f"{prefix}{i:03d}")
        diff = img - prev
        prev = img
        diff = normalize(diff)
        plt.figure(figsize=(10,10))
        plt.title(f"{prefix}{i:03d}", fontsize=20)
        plt.imshow(diff)

In [ ]:
prefix = "moving_circle_"
show_motion(prefix, frames=2)

In [ ]:
def RGB_to_YCoCg(RGB_frame):
    R, G, B = RGB_frame[:,:,0], RGB_frame[:,:,1], RGB_frame[:,:,2]
    YCoCg_frame = np.empty_like(RGB_frame)
    YCoCg_frame[:,:,0] =  R/4 + G/2 + B/4 
    YCoCg_frame[:,:,1] =  R/2       - B/2
    YCoCg_frame[:,:,2] = -R/4 + G/2 - B/4
    return YCoCg_frame

In [ ]:
frame_0 = load_frame(prefix + "000")
frame_1 = load_frame(prefix + "001")
write_frame(frame_1-frame_0, "difference")

In [ ]:
frame_0_Y = RGB_to_YCoCg(frame_0)[:,:,0]
frame_1_Y = RGB_to_YCoCg(frame_1)[:,:,0]

In [ ]:
plt.figure(figsize=(10,10))
plt.title("frame_0_Y", fontsize=20)
plt.imshow(frame_0_Y, cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("frame_1_Y", fontsize=20)
plt.imshow(frame_1_Y, cmap='gray')
plt.show()

In [ ]:
# Plot the first part (from pixel 0 to 39) of the horizontal line (row 35) of the luminance.
plt.plot(frame_0_Y[35, 0:40])
#plt.show()
plt.plot(frame_1_Y[35, 0:40], linestyle=':')
plt.savefig("movement.svg")
plt.savefig("movement.pdf")
plt.show()

In [ ]:
LL_0_haar, (LH_0_haar, HL_0_haar, HH_0_haar) = pywt.dwt2(frame_0_Y, 'haar')
iL_0_haar = pywt.idwt2([LL_0_haar, (None, None, None)], 'haar')
LL_1_haar, (LH_1_haar, HL_1_haar, HH_1_haar) = pywt.dwt2(frame_1_Y, 'haar')
iL_1_haar = pywt.idwt2([LL_1_haar, (None, None, None)], 'haar')

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iL_0_haar", fontsize=20)
plt.imshow(iL_0_haar, cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iL_1_haar", fontsize=20)
plt.imshow(iL_1_haar, cmap='gray')
plt.show()

In [ ]:
plt.title("Coefs in iL (Haar)")
plt.plot(iL_0_haar[35, 0:40])
plt.plot(iL_1_haar[35, 0:40], linestyle=':')
plt.savefig("haar_iL.svg")
plt.savefig("haar_iL.pdf")
plt.show()

In [ ]:
LL_0_db5, (LH_0_db5, HL_0_db5, HH_0_db5) = pywt.dwt2(frame_0_Y, 'db5')
iL_0_db5 = pywt.idwt2([LL_0_db5, (None, None, None)], 'db5')
LL_1_db5, (LH_1_db5, HL_1_db5, HH_1_db5) = pywt.dwt2(frame_1_Y, 'db5')
iL_1_db5 = pywt.idwt2([LL_1_db5, (None, None, None)], 'db5')

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iL_0_db5", fontsize=20)
plt.imshow(iL_0_db5, cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iL_1_db5", fontsize=20)
plt.imshow(iL_1_db5, cmap='gray')
plt.show()

In [ ]:
plt.title("Coefs in iL (Daubechies 5)")
plt.plot(iL_0_db5[35, 0:40])
plt.plot(iL_1_db5[35, 0:40], linestyle=':')
plt.savefig("db5_iL.svg")
plt.savefig("db5_iL.pdf")
plt.show()

In [ ]:
LL_0_bior35, (LH_0_bior35, HL_0_bior35, HH_0_bior35) = pywt.dwt2(frame_0_Y, 'bior3.5')
iL_0_bior35 = pywt.idwt2([LL_0_bior35, (None, None, None)], 'bior3.5')
LL_1_bior35, (LH_1_bior35, HL_1_bior35, HH_1_bior35) = pywt.dwt2(frame_1_Y, 'bior3.5')
iL_1_bior35 = pywt.idwt2([LL_1_bior35, (None, None, None)], 'bior3.5')

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iL_0_bior35", fontsize=20)
plt.imshow(iL_0_bior35, cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iL_1_bior35", fontsize=20)
plt.imshow(iL_1_bior35, cmap='gray')
plt.show()

In [ ]:
plt.title("Coefs in iL (Biorthogonal 3.5)")
plt.plot(iL_0_bior35[35, 0:40])
plt.plot(iL_1_bior35[35, 0:40], linestyle=':')
plt.savefig("bior35_iL.svg")
plt.savefig("bior35_iL.pdf")
plt.show()

In [ ]:
iH_0_haar = pywt.idwt2([None,(LH_0_haar, HL_0_haar, HH_0_haar)], 'haar')
iH_1_haar = pywt.idwt2([None,(LH_1_haar, HL_1_haar, HH_1_haar)], 'haar')

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iH_0_haar", fontsize=20)
plt.imshow(iH_0_haar, cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iH_1_haar", fontsize=20)
plt.imshow(iH_1_haar, cmap='gray')
plt.show()

In [ ]:
plt.title("Coefs in iH (Haar)")
plt.plot(iH_0_haar[35, 0:40])
plt.plot(iH_1_haar[35, 0:40], linestyle=':')
plt.savefig("haar_iH.svg")
plt.savefig("haar_iH.pdf")
plt.show()

In [ ]:
iH_0_db5 = pywt.idwt2([None,(LH_0_db5, HL_0_db5, HH_0_db5)], 'db5')
iH_1_db5 = pywt.idwt2([None,(LH_1_db5, HL_1_db5, HH_1_db5)], 'db5')

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iH_0_db5", fontsize=20)
plt.imshow(iH_0_db5, cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iH_1_db5", fontsize=20)
plt.imshow(iH_1_db5, cmap='gray')
plt.show()

In [ ]:
plt.title("Coefs in iH (db5)")
plt.plot(iH_0_db5[35, 0:40])
plt.plot(iH_1_db5[35, 0:40], linestyle=':')
plt.savefig("db5_iH.svg")
plt.savefig("db5_iH.pdf")
plt.show()

In [ ]:
iH_0_bior35 = pywt.idwt2([None,(LH_0_bior35, HL_0_bior35, HH_0_bior35)], 'bior3.5')
iH_1_bior35 = pywt.idwt2([None,(LH_1_bior35, HL_1_bior35, HH_1_bior35)], 'bior3.5')

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iH_0_bior35", fontsize=20)
plt.imshow(iH_0_bior35, cmap='gray')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.title("iH_1_bior35", fontsize=20)
plt.imshow(iH_1_bior35, cmap='gray')
plt.show()

In [ ]:
plt.title("Coefs in iH (bior35)")
plt.plot(iH_0_bior35[35, 0:40])
plt.plot(iH_1_bior35[35, 0:40], linestyle=':')
plt.savefig("bior35_iH.svg")
plt.savefig("bior35_iH.pdf")
plt.show()